In [1]:
import json, os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import Mask2Base64

In [2]:
def crop_blocks(data_dir, res_folder, ann_folder='ann', img_folder='img', echo=False):
    ann_files = os.listdir(os.path.join(data_dir, ann_folder))
    for fname in ann_files:
        if echo:
            print("\n", fname)
        img_name = fname.split('.')[0]
        image = cv2.imread(os.path.join(data_dir, img_folder, img_name + '.png'))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        with open(os.path.join(data_dir, ann_folder, fname), 'r') as json_file:
            data = json.load(json_file)
            if data['objects']:
                for rod in data['objects']:
                    if echo:
                        print(rod['id'])
                    mask = np.array(Mask2Base64.base64_2_mask(rod['bitmap']['data']),dtype=int)
                    x_min = rod['bitmap']['origin'][0]
                    y_min = rod['bitmap']['origin'][1]
                    x_max = x_min + mask.shape[1]
                    y_max = y_min + mask.shape[0]
                    
                    cropped_image = image[y_min:y_max,x_min:x_max]
                    file_name = os.path.join(res_folder, img_name + '_ROD_' + str(rod['id']) + '.png')
                    cv2.imwrite(file_name, cropped_image)

In [3]:
DATASETS = 'quality_segmentation/appropriate'
CROPPED = 'cropped'
crop_blocks(DATASETS, CROPPED)